In [ ]:
import numpy as np
import math
import sys
sys.path.append('../../python/')  
from periphery import logicGate
from periphery import constant
print(constant.INV)

In [ ]:
class SubArray:
    def __init__(self, tech, param, config,relaxArrayCellWidth,relaxArrayCellHeight):
        self.tech = tech
        self.param = param
        self.config = config
        self.initialized = False
        self.relaxArrayCellWidth = relaxArrayCellWidth
        self.relaxArrayCellHeight = relaxArrayCellHeight

    def initialize(self, numRow, numCol, unitWireRes):
        self.numRow = numRow
        self.numCol = numCol
        self.unitWireRes = unitWireRes

        feature_size = self.tech['featureSize']
        min_cell_height = self.tech['MAX_TRANSISTOR_HEIGHT'];  #set real layout cell height
        min_cell_width =  (self.tech['MIN_GAP_BET_GATE_POLY'] + self.tech['POLY_WIDTH']) * 2;  #set real layout cell width
        siolation_region = self.tech['MIN_POLY_EXT_DIFF'] * 2 + self.tech['MIN_GAP_BET_FIELD_POLY']

        #############################################
        # to be done: for the technology below 14nm, there will be some tuning for the cell width and height and islation region
        #############################################

        if self.cell.memCellType == 'SRAM':
            if self.relaxArrayCellWidth:
                width = max(self.config.widthInFeatureSize, min_cell_width)
                height = max(self.config.heightInFeatureSize, min_cell_height)
                self.lengthRow = numCol * width * feature_size
                self.lengthCol = numRow * height * feature_size
            else:
                self.lengthRow = numCol * self.config.widthInFeatureSize * feature_size
            if self.relaxArrayCellHeight:
                self.lengthCol = numRow * height * feature_size
            else:
                self.lengthCol = numRow * self.config.heightInFeatureSize * feature_size

        self.arraywidthunit = self.config.widthInFeatureSize * feature_size
        self.arrayheightunit = numRow * self.config.heightInFeatureSize * feature_size
        ###finish setting array size

        # elif self.cell.memCellType in ['RRAM', 'FeFET']:
        #     width = self.cell.widthInFeatureSize
        #     height = self.cell.heightInFeatureSize

        #     if self.cell.accessType == 'CMOS_access':
        #         width = max(width, min_cell_width * 2)
        #         height = max(height, min_cell_height)
        #     else:
        #         width = max(width * feature_size, min_cell_width * 2 * feature_size)
        #         height = max(height * feature_size, min_cell_height * feature_size)

        #     self.lengthRow = numCol * width
        #     self.lengthCol = numRow * height

        # cap and resistance calculation
        self.capRow1 = self.lengthRow * 0.2e-15 / 1e-6      # BL for 1T1R, WL for Cross-point and SRAM
        self.capRow2 = self.capRow1                         # WL for 1T1R
        self.capCol = self.lengthCol * 0.2e-15 / 1e-6
        self.resRow = self.lengthRow * self.config.Metal1_unitwireresis
        self.resCol = self.lengthCol * self.config.Metal0_unitwireresis

        #start to initialize the subarray modules
        if self.cell.memCellType == 'SRAM':
            #firstly calculate the CMOS resistance and capacitance
            width_access_cmos = self.config.widthAccessCMOS
            scale_factor = 2 if feature_size <= 14e-9 else 1
            effective_width = width_access_cmos * scale_factor * feature_size
            # resCellAccess = logic_gate.calculate_on_resistance(cell.widthAccessCMOS * ((tech.featureSize <= 14*1e-9)? 2:1) * tech.featureSize, NMOS, inputParameter.temperature, tech);
            resCellAccess = logic_gate.calculate_on_resistance(effective_width,constant.NMOS,self.config.temperature, self.tech)
            capCellAccess = logic_gate.calculate_drain_cap(effective_width,constant.NMOS, self.tech['MAX_TRANSISTOR_HEIGHT'] * feature_size, self.tech)
             
		    cell.capSRAMCell = capCellAccess + CalculateDrainCap(cell.widthSRAMCellNMOS * ((tech.featureSize <= 14*1e-9)? 2:1) * tech.featureSize, NMOS, MAX_TRANSISTOR_HEIGHT * tech.featureSize, tech) 
						+ CalculateDrainCap(cell.widthSRAMCellPMOS * ((tech.featureSize <= 14*1e-9)? 2:1) * tech.featureSize, PMOS, MAX_TRANSISTOR_HEIGHT * tech.featureSize, tech) 
						+ CalculateGateCap(cell.widthSRAMCellNMOS * ((tech.featureSize <= 14*1e-9)? 2:1) * tech.featureSize, tech) + CalculateGateCap(cell.widthSRAMCellPMOS * ((tech.featureSize <= 14*1e-9)? 2:1) * tech.featureSize, tech);
        self.initialized = True


In [ ]:
class SarADC:
    def __init__(self, tech, param):
        self.tech = tech
        self.param = param
        self.initialized = False

    def initialize(self, num_col, level_output, clk_freq, num_read_cell):
        self.num_col = num_col
        self.level_output = level_output
        self.clk_freq = clk_freq
        self.num_read_cell = num_read_cell
        self.width_nmos = self.tech['featureSize'] * self.tech['MIN_NMOS_SIZE']
        self.width_pmos = self.tech['pnSizeRatio'] * self.tech['featureSize'] * self.tech['MIN_NMOS_SIZE']
        self.initialized = True

    def calculate_unit_area(self):
        h_nmos = self.tech['featureSize'] * self.tech['MAX_TRANSISTOR_HEIGHT']
        w_nmos = self.width_nmos
        h_pmos = self.tech['featureSize'] * self.tech['MAX_TRANSISTOR_HEIGHT']
        w_pmos = self.width_pmos
        self.area_unit = (h_nmos * w_nmos) * (269 + (math.log2(self.level_output) - 1) * 109) + \
                         (h_pmos * w_pmos) * (209 + (math.log2(self.level_output) - 1) * 73)

    def calculate_area(self, height_array, width_array, option='NONE'):
        if option == 'NONE':
            self.area = self.area_unit * self.num_col
            if width_array:
                self.width = width_array
                self.height = self.area / width_array
            elif height_array:
                self.height = height_array
                self.width = self.area / height_array
            else:
                raise ValueError("No width or height assigned for layout.")
        # MagicLayout or OverrideLayout can be implemented as needed

    def calculate_latency(self, num_read):
        self.read_latency = (math.log2(self.level_output) + 1) * num_read * 1/ self.clk_freq

    def calculate_power(self, column_resistance_list, num_read):
        self.read_dynamic_energy = 0
        for res in column_resistance_list:
            self.read_dynamic_energy += self.get_column_power(res)
        self.read_dynamic_energy *= num_read

    def get_column_power(self, column_res):
        # convert the column resistance to 0.5 read voltage
        col_res = column_res * 0.5 / self.param['readVoltage']
        tech_node = self.param['technode']
        level = math.log2(self.level_output)

        #model is based on HP and LP roadmap
        # base = A + B * level
        # exp_term = C * exp(D * log10(col_res))
        #A, B, C, D are different for each technology node
        if self.param['deviceroadmap'] == 1:  # HP
            if tech_node == 130:
                base = (6.4806 * level + 49.047) * 1e-6
                exp_term = 0.207452 * math.exp(-2.367 * math.log10(col_res))
            elif tech_node == 90:
                base = (4.3474 * level + 31.782) * 1e-6
                exp_term = 0.1649 * math.exp(-2.345 * math.log10(col_res))
            elif tech_node == 65:
                base = (2.9503 * level + 22.047) * 1e-6
                exp_term = 0.128483 * math.exp(-2.321 * math.log10(col_res))
            elif tech_node == 45:
                base = (2.1843 * level + 11.931) * 1e-6
                exp_term = 0.097754 * math.exp(-2.296 * math.log10(col_res))
            elif tech_node == 32:
                base = (1.0157 * level + 7.6286) * 1e-6
                exp_term = 0.083709 * math.exp(-2.313 * math.log10(col_res))
            elif tech_node == 22:
                base = (0.7213 * level + 3.3041) * 1e-6
                exp_term = 0.084273 * math.exp(-2.311 * math.log10(col_res))
            elif tech_node == 14:
                base = (0.4710 * level + 1.9529) * 1e-6
                exp_term = 0.060584 * math.exp(-2.311 * math.log10(col_res))
            elif tech_node == 10:
                base = (0.3076 * level + 1.1543) * 1e-6
                exp_term = 0.049418 * math.exp(-2.311 * math.log10(col_res))
            else:  # 7nm
                base = (0.2008 * level + 0.6823) * 1e-6
                exp_term = 0.040310 * math.exp(-2.311 * math.log10(col_res))
        else:  # LP
            if tech_node == 130:
                base = (8.4483 * level + 65.243) * 1e-6
                exp_term = 0.16938 * math.exp(-2.303 * math.log10(col_res))
            elif tech_node == 90:
                base = (5.9869 * level + 37.462) * 1e-6
                exp_term = 0.144323 * math.exp(-2.303 * math.log10(col_res))
            elif tech_node == 65:
                base = (3.7506 * level + 25.844) * 1e-6
                exp_term = 0.121272 * math.exp(-2.303 * math.log10(col_res))
            elif tech_node == 45:
                base = (2.1691 * level + 16.693) * 1e-6
                exp_term = 0.100225 * math.exp(-2.303 * math.log10(col_res))
            elif tech_node == 32:
                base = (1.1294 * level + 8.8998) * 1e-6
                exp_term = 0.079449 * math.exp(-2.297 * math.log10(col_res))
            elif tech_node == 22:
                base = (0.538 * level + 4.3753) * 1e-6
                exp_term = 0.072341 * math.exp(-2.303 * math.log10(col_res))
            elif tech_node == 14:
                base = (0.3132 * level + 2.5681) * 1e-6
                exp_term = 0.061085 * math.exp(-2.303 * math.log10(col_res))
            elif tech_node == 10:
                base = (0.1823 * level + 1.5073) * 1e-6
                exp_term = 0.05158 * math.exp(-2.303 * math.log10(col_res))
            else:  # 7nm
                base = (0.1061 * level + 0.8847) * 1e-6
                exp_term = 0.043555 * math.exp(-2.303 * math.log10(col_res))

        # consider temperature effect, power increase by 0.13% per degree Celsius
        column_power = (base + exp_term) * (1 + 1.3e-3 * (self.param['temp'] - 300))
        column_energy = column_power * (level + 1) * 1/ self.clk_freq  # Energy = Power * Time
        return column_energy
